1. Business Understanding

The "Hotels Reservation" dataset was uploaded by "Ahsan Raza" on kaggle in an effort to discover patterns in online-booking tendencies of hotel customers, specifically if a customer would honor the reservation or if they would cancel it. The data was collected from a single country which cannot be disclosed for privacy reasons as stated by Ahsan in the Kaggle discussion posts.

The datasets description states there have been major issues in the Hotel industry in regard to consumer online booking activity. The convenience and low to little cost of cancelling an online-booking has led to a significant change in consumer behavior. A large number of cancellations include changes of plans, scheduling conflicts, etc. Ahsan describes this convenient cancellation feature as "beneficial to hotel guests, but less desirable and possible revenue diminishing for hotels".

One can use this dataset to predict if a consumer will honor their reservation or not. Parties that may be interested in the classification result may be those employed in Hotel Management, or management from any industry that deals with an online reservation format such as restaurants, resorts, or motels. A hotel manager will be able to gain insight as to why certain customers may be more prone to online cancelling.

The dataset contains 19 features, 13 numerical and 6 categorical.


1.1 Measure of Success
So, what defines a successful machine learning algorithm?
Due to the non-critical nature of hotel cancellations, it is safe to assume that a good model should be able to predict more accurately than random occurrence (50/50). For a business with a more aggressive strategy to produce profit, baseline of 80% for the classification rate may be ideal.